## Asilomar 2022 Signature velocity QC
Use thresholds on beam correlation and beam amplitude to discard bad beam velocities.


In [1]:
# Imports
import os
import glob
import numpy as np 
import pandas as pd
import xarray as xr
from scipy.io import loadmat
from scipy.signal import detrend
from datetime import timedelta as TD
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib widget
from PyPDF2 import PdfWriter
from roxsi_pyfuns.preprocess import signature_preprocess as rpsp
import warnings
warnings.filterwarnings("ignore")
import re

def natural_sort(l): 
    """ 
    Natural sorting function borrowed from
    https://stackoverflow.com/questions/4836710/is-there-a-built-in-function-for-string-natural-sort
    """
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

In [6]:
# Paths etc
ser = '103110' # Signature ADCP serial number
rootdir = r'/media/mikapm/T7 Shield/ROXSI/Asilomar2022/SmallScaleArray'
sigdir = os.path.join(rootdir, 'Signatures')
outdir = os.path.join(sigdir, 'Level1', ser)
datadir = os.path.join(sigdir, 'raw', ser)
fn_minfo = os.path.join(rootdir, 'Asilomar_SSA_2022_mooring_info.xlsx')
# Read bathymetry netcdf file
bathydir = os.path.join(rootdir, 'Bathy')
fn_bathy = os.path.join(bathydir, 'Asilomar_2022_SSA_bathy_updated_50cm.nc')
print('Reading bathy file')
dsb = xr.decode_cf(xr.open_dataset(fn_bathy, decode_coords='all'))
# Atmospheric pressure file
fn_patm = os.path.join(rootdir, 'noaa_atm_pressure.csv')
print('Reading atm pressure file')
dfa = pd.read_csv(fn_patm, parse_dates=['time']).set_index('time')

# Initialize ADCP reader class
print('Initializing ADCP class')
adcp = rpsp.ADCP(datadir=datadir, ser=ser, mooring_info=fn_minfo, 
                 outdir=outdir, patm=dfa, bathy=None)
# Save all datasets for the same date in list for concatenating
dsv_daily = [] # Velocities and 1D (eg AST) data
dse_daily = [] # Echogram data

# Array of dates to be included
dates = pd.date_range('2022-06-25', '2022-07-21', freq='1d').strftime('%Y-%m-%d')
for datestr in dates:
    # print(f'Date: {datestr}')
    # Output netcdf filename, check if file already exists
    fn_out = os.path.join(outdir, f'sig_{ser}_vel_amp_corr_{pd.Timestamp(datestr).strftime("%Y%m%d")}.nc')
    if os.path.isfile(fn_out):
        continue
    # If netcdf file doesn't exist, read velocities etc. and save to netcdf
    ds_list = [] # List for appending daily datasets for concatenating
    # Loop over raw .mat files and save daily data as netcdf
    for i,fn_mat in tqdm(enumerate(natural_sort(adcp.fns))):
        # Check if daily netcdf files already exist
        times_mat, times = adcp.read_mat_times(fn_mat=fn_mat)
        date0 = str(times[0].date()) # Date of first timestamp
        # Check if we can move on to next date
        if pd.Timestamp(date0) > pd.Timestamp(datestr):
            print(f'Moving on to next date. date0: {date0}, datestr:{datestr} ...')
            print(f'fn: {fn_mat}')
            break
        date1 = str(times[-1].date()) # Date of last timestamp
        if date0 != datestr and date1 != datestr:
            # print(f'date0={date0}, date1={date1}')
            continue
        # L5 mat files seem to align exactly with dates,
        # so add 1h to date1 if ser == 103206
        if ser == '103206':
            date1 = str((times[-1] + TD(hours=1)).date())
        # Check if date1 is before dataset starttime
        if pd.Timestamp(times[-1]) < pd.Timestamp(adcp.t0):
            print('.mat file endtime {} before dataset starttime {}'.format(
                pd.Timestamp(times[-1]), pd.Timestamp(adcp.t0)))
            continue
        # Read .mat file to dict
        mat = loadmat(fn_mat)
        # Read beam vel, amp. and correlation from .mat file into dataset
        dsi = adcp.ampcorr2ds(mat)
        # Append dataset to list for concatenating
        ds_list.append(dsi)
    # Concatenate datasets for current date
    if len(ds_list) > 0:
        print(f'Concatenating datasets for {datestr}')
        dsc = xr.concat(ds_list, dim='time')
        # Crop dataset to current date only
        dsc = dsc.sel(time=datestr)
        # Save to netcdf
        print(f'Saving {datestr} serial number {ser} to netCDF')
        dsc.to_netcdf(fn_out)
print('Done.')

Reading bathy file
Reading atm pressure file
Initializing ADCP class


0it [00:00, ?it/s]

Moving on to next date. date0: 2022-07-08, datestr:2022-07-07 ...
fn: /media/mikapm/T7 Shield/ROXSI/Asilomar2022/SmallScaleArray/Signatures/raw/103110/S103110A003_AS22_SS_0031_1.mat
Concatenating datasets for 2022-07-07
Saving 2022-07-07 serial number 103110 to netCDF


0it [00:00, ?it/s]

Moving on to next date. date0: 2022-07-09, datestr:2022-07-08 ...
fn: /media/mikapm/T7 Shield/ROXSI/Asilomar2022/SmallScaleArray/Signatures/raw/103110/S103110A003_AS22_SS_0033_2.mat
Concatenating datasets for 2022-07-08
Saving 2022-07-08 serial number 103110 to netCDF


...

Moving on to next date. date0: 2022-07-20, datestr:2022-07-19 ...
fn: /media/mikapm/T7 Shield/ROXSI/Asilomar2022/SmallScaleArray/Signatures/raw/103110/S103110A003_AS22_SS_0056_2.mat
Concatenating datasets for 2022-07-19
Saving 2022-07-19 serial number 103110 to netCDF


0it [00:00, ?it/s]

Moving on to next date. date0: 2022-07-21, datestr:2022-07-20 ...
fn: /media/mikapm/T7 Shield/ROXSI/Asilomar2022/SmallScaleArray/Signatures/raw/103110/S103110A003_AS22_SS_0058_2.mat
Concatenating datasets for 2022-07-20
Saving 2022-07-20 serial number 103110 to netCDF


0it [00:00, ?it/s]

3 duplicates found in time_arr
duplicates:  [datetime.datetime(2022, 7, 21, 8, 56, 1, 625805)
 datetime.datetime(2022, 7, 21, 8, 56, 1, 875803)
 datetime.datetime(2022, 7, 21, 8, 56, 2, 125792)]
after removal of dupl.: 0 left
3 duplicates found in tb5
duplicates:  [datetime.datetime(2022, 7, 21, 8, 56, 1, 813291)
 datetime.datetime(2022, 7, 21, 8, 56, 2, 63290)
 datetime.datetime(2022, 7, 21, 8, 56, 2, 313298)]
after removal of dupl.: 0 left
6 duplicates found in tb5
duplicates:  [datetime.datetime(2022, 7, 21, 12, 56, 22, 813395)
 datetime.datetime(2022, 7, 21, 12, 56, 25, 313301)
 datetime.datetime(2022, 7, 21, 12, 56, 27, 813499)
 datetime.datetime(2022, 7, 21, 12, 56, 30, 313395)
 datetime.datetime(2022, 7, 21, 12, 56, 32, 813401)
 datetime.datetime(2022, 7, 21, 12, 58, 3, 563402)]
after removal of dupl.: 0 left
Concatenating datasets for 2022-07-21
Saving 2022-07-21 serial number 103110 to netCDF
Done.


Perform QC based on threshold on beam correlation and amplitude. Use standard thresholds recommended by Nortek: Correlation threshold = 50%, amplitude threshold = 30 dB.

In [7]:
mids = {'103206':'L5', '103088':'C1', '103094':'C3', '103110':'C6'} # Mooring IDs
ser = '103110' # Signature serial number
mid = mids[ser]
rootdir = r'/media/mikapm/T7 Shield/ROXSI/Asilomar2022/SmallScaleArray'
sigdir = os.path.join(rootdir, 'Signatures')
outdir = os.path.join(sigdir, 'Level1', ser)
datadir = os.path.join(sigdir, 'Level1', ser)
corr_thresh = 50 # Beam correlation lower threshold
amp_thresh = 30 # Beam amplitude lower threshold
dates = pd.date_range('2022-06-25', '2022-07-21', freq='1d').strftime('%Y%m%d')

# Iterate over dates and assign amplitude and correlation variables to existing nc files
for datestr in tqdm(dates):
    # Define new output filename
    fn_out = os.path.join(outdir, f'Asilomar_SSA_Sig_Vel_{ser}_{datestr}_v2.nc')
    # Check if output file already exists
    if os.path.isfile(fn_out):
        continue
    if datestr == '20220703' and ser == '103088':
        continue
    # Read new file with ampl + corr.
    fn_vac = os.path.join(datadir, f'sig_{ser}_vel_amp_corr_{datestr}.nc')
    ds_vac = xr.open_dataset(fn_vac, engine='netcdf4')
    # Read original netcdf file w/o ampl + corr
    fn_org = os.path.join(datadir, f'Asilomar_SSA_L1_Sig_Vel_{mid}_{datestr}.nc')
    ds_org = xr.open_dataset(fn_org, decode_coords='all')
    if len(ds_org.time.values) != len(ds_vac.time.values):
        # Reindex vac dataset to original time index
        # ds_vac = ds_vac.sel(time=slice(ds_org.time.values[0], ds_org.time.values[-1]))
        ds_vac = ds_vac.reindex(time=ds_org.time.values, method='nearest', tolerance='10ms')
    # Assign variables w/ CF attributes
    ds_org['ampB1'] = (['time', 'range'], ds_vac.ab1.values, {'units':'dB', 
                                                              'standard_name':'signal_intensity_from_multibeam_acoustic_doppler_velocity_sensor_in_sea_water',
                                                              'long_name':'Beam 1 amplitude',
                                                              },
                       )
    ds_org['ampB2'] = (['time', 'range'], ds_vac.ab2.values, {'units':'dB', 
                                                              'standard_name':'signal_intensity_from_multibeam_acoustic_doppler_velocity_sensor_in_sea_water',
                                                              'long_name':'Beam 2 amplitude',
                                                              },
                       )
    ds_org['ampB3'] = (['time', 'range'], ds_vac.ab3.values, {'units':'dB', 
                                                              'standard_name':'signal_intensity_from_multibeam_acoustic_doppler_velocity_sensor_in_sea_water',
                                                              'long_name':'Beam 3 amplitude',
                                                              },
                       )
    ds_org['ampB4'] = (['time', 'range'], ds_vac.ab4.values, {'units':'dB', 
                                                              'standard_name':'signal_intensity_from_multibeam_acoustic_doppler_velocity_sensor_in_sea_water',
                                                              'long_name':'Beam 4 amplitude',
                                                              },
                       )
    ds_org['ampB5'] = (['time', 'range'], ds_vac.ab5.values, {'units':'dB', 
                                                              'standard_name':'signal_intensity_from_multibeam_acoustic_doppler_velocity_sensor_in_sea_water',
                                                              'long_name':'Beam 5 amplitude',
                                                              },
                       )
    ds_org['corrB1'] = (['time', 'range'], ds_vac.cb1.values, {'units':'', 
                                                               'standard_name':'beam_consistency_indicator_from_multibeam_acoustic_doppler_velocity_profiler_in_sea_water',
                                                               'long_name':'Beam 1 correlation',
                                                               },
                       )
    ds_org['corrB2'] = (['time', 'range'], ds_vac.cb2.values, {'units':'', 
                                                               'standard_name':'beam_consistency_indicator_from_multibeam_acoustic_doppler_velocity_profiler_in_sea_water',
                                                               'long_name':'Beam 2 correlation',
                                                               },
                       )
    ds_org['corrB3'] = (['time', 'range'], ds_vac.cb3.values, {'units':'', 
                                                               'standard_name':'beam_consistency_indicator_from_multibeam_acoustic_doppler_velocity_profiler_in_sea_water',
                                                               'long_name':'Beam 3 correlation',
                                                               },
                       )
    ds_org['corrB4'] = (['time', 'range'], ds_vac.cb4.values, {'units':'', 
                                                               'standard_name':'beam_consistency_indicator_from_multibeam_acoustic_doppler_velocity_profiler_in_sea_water',
                                                               'long_name':'Beam 4 correlation',
                                                               },
                       )
    ds_org['corrB5'] = (['time', 'range'], ds_vac.cb5.values, {'units':'', 
                                                               'standard_name':'beam_consistency_indicator_from_multibeam_acoustic_doppler_velocity_profiler_in_sea_water',
                                                               'long_name':'Beam 5 correlation',
                                                               },
                       )
    # Save to new netcdf
    ds_org.to_netcdf(fn_out)

  0%|          | 0/27 [00:00<?, ?it/s]

Plot amplitude + correlation + beam vel.

In [ ]:
plt.clf()
plt.close()
ser = '103110' # Signature serial number
mids = {'103206':'L5', '103088':'C1', '103094':'C3', '103110':'C6'} # Mooring IDs
sig_mids = {'C1':'SO', 'C3':'SM', 'C6':'SI', 'L1':'SS', 'L5':'SN'}
mid = mids[ser]
rootdir = r'/media/mikapm/T7 Shield/ROXSI/Asilomar2022/SmallScaleArray'
sigdir = os.path.join(rootdir, 'Signatures')
outdir = os.path.join(sigdir, 'Level1', ser)
figdir = os.path.join(sigdir, 'Level1', ser, 'qc_fig')
if not os.path.isdir(figdir):
    os.mkdir(figdir)
datadir = os.path.join(sigdir, 'Level1', ser)
corr_thresh = 75 # Beam correlation lower threshold
amp_thresh = 40 # Beam amplitude lower threshold
dates = pd.date_range('2022-07-05', '2022-07-14', freq='1d').strftime('%Y%m%d')

for datestr in dates:
    # datestr = '20220707'
    print(f'Date: {datestr}')
    # Read new netcdf file
    fn = os.path.join(datadir, f'Asilomar_SSA_Sig_Vel_{ser}_{datestr}_v2.nc')
    ds = xr.open_dataset(fn, decode_coords='all')

    # Iterate over hours and bins and estimate beam velocity spectra
    hours = pd.date_range(pd.Timestamp(datestr), pd.Timestamp(datestr)+pd.Timedelta(days=1), freq='1h')
    for t0,t1 in tqdm(zip(hours[:-1], hours[1:])):
        # Take out hourly segment
        seg = ds.sel(time=slice(t0,t1)).copy()
        # Make masks for keeping only good values
        valid = (seg.range < (0.9 * seg.ASTd.min(dim='time').item())) # Valid range values below surface
        good = {'B1': ((seg.ampB1>amp_thresh) & (seg.corrB1>corr_thresh)), # Good Beam1 values based on amp&corr
                'B2': ((seg.ampB2>amp_thresh) & (seg.corrB2>corr_thresh)), # Good Beam2 values based on amp&corr
                'B3': ((seg.ampB3>amp_thresh) & (seg.corrB3>corr_thresh)), # Good Beam3 values based on amp&corr
                'B4': ((seg.ampB4>amp_thresh) & (seg.corrB4>corr_thresh)), # Good Beam4 values based on amp&corr
                'B5': ((seg.ampB5>amp_thresh) & (seg.corrB5>corr_thresh)), # Good Beam5 values based on amp&corr
                }
        # np.isnan(ds.vB1.where((ds.ampB1>40) & (ds.corrB1>50))).plot(ax=ax, x='time', y='range')
        for b in ['vB1', 'vB2', 'vB3', 'vB4', 'vB5']:
            # Filename for merged pdf
            fn_merged = os.path.join(figdir, f'merged_{ser}_{b}_{datestr}.pdf')
            if os.path.isfile(fn_merged):
                continue
            # Output figure filename
            fn_fig = os.path.join(figdir, f'{ser}_{b}_{t0.strftime("%Y%m%d_%H%M")}.pdf')
            if os.path.isfile(fn_fig):
                continue
            fig, axes = plt.subplots(figsize=(12,6), nrows=2, constrained_layout=True, sharex=True)
            # Select only valid data
            good_vb = seg[b].where((valid & good['B1'] & good['B2'] & good['B3'] & good['B4'])).copy()
            # Only use range bins with less than 10% dropouts/discarded values
            good_ranges = (np.isnan(good_vb).sum(axis=0) < len(good_vb.time.values)*0.1)
            # Check if any good range bins, else skip hour
            if np.sum(good_ranges) == 0:
                continue
            good_vb = good_vb.sel(range=(good_ranges.values))
            # Plot beam velocity profile on top (downsample for plot)
            good_vb.resample(time='2s').mean().plot(ax=axes[0], x='time', y='range', cbar_kwargs={"pad":0.005})
            # Also plot AST range
            AST_mean = seg.ASTd.mean(dim='time').item()
            AST_dt = detrend(seg.ASTd.interpolate_na(dim='time').bfill(dim='time').ffill(dim='time'))
            axes[0].plot(seg.time, (AST_dt+AST_mean), c='k')
            # Timeseries at different range bins
            if ser == '103094':
                ranges = [3.5, 2.25, 1]
            else:
                ranges = [5.5, 3.5, 1.5]
            cs = ['k', 'C0', 'C1'] # Colors
            for ri,r in enumerate(ranges):
                axes[0].axhline(r, ls='--', lw=0.75, c=cs[ri], label=f'range={r}m')
                timeseries = good_vb.sel(range=r, method='nearest').interpolate_na(dim='time').copy()
                (timeseries+0.35).plot(ax=axes[1], alpha=0.75, c=cs[ri], label=f'range={r}m',)
            axes[0].set_xlabel(None)
            axes[0].set_ylim([0, seg.ASTd.max()+0.25])
            axes[0].set_title(f'{ser} Signature ({sig_mids[mid]}) Beam {b[-1]} velocity for {t0}-{t1.time()}')
            axes[1].set_title(None)
            axes[1].legend(ncols=len(ranges))
            # plt.show()
            # Save fig
            plt.savefig(fn_fig, bbox_inches='tight', dpi=300)
            plt.close()

    # Merge individual pdfs into one
    for b in ['vB1', 'vB2', 'vB3', 'vB4', 'vB5']:
        # Filename for merged pdf
        fn_merged = os.path.join(figdir, f'merged_{ser}_{b}_{datestr}.pdf')
        if os.path.isfile(fn_merged):
            continue
        # List all pdf files for current beam
        fns_pdf = sorted(glob.glob(os.path.join(figdir, f'{ser}_{b}_*.pdf')))
        # Initialize pdf merging class
        merger = PdfWriter()
        # print('Merging individual pdf files ...')
        # Append individual pdf files
        for pdf in fns_pdf:
            merger.append(pdf)

        merger.write(fn_merged)
        merger.close()
        # Remove individual pdf files
        # print('Removing individual pdf files ...')
        for fn in fns_pdf:
            os.remove(fn)


Compare ADCP beam velocity statistics to linear wave theory predictions.